In [1]:
import pandas as pd

In [2]:
ddi_subset = pd.read_csv("../mined_data/DDI_subset.csv")
dpi_subset = pd.read_csv("../mined_data/DPI_subset.csv")
ppi_subset = pd.read_csv("../mined_data/PPI_subset.csv")

In [6]:
len(ddi_subset), len(dpi_subset), len(ppi_subset)

(308, 331, 210)

In [17]:
from pathlib import Path
drug_data_path = Path("../background_information_data/drug_data")
protein_data_path = Path("../background_information_data/protein_data")

In [53]:
triples = []
for row in dpi_subset.itertuples():
    with Path(drug_data_path / f"{row.drug_name}.txt").open("r") as drug_data_file:
        drug_data = max(drug_data_file.readlines(), key=len)

    with Path(protein_data_path / f"{row.protein_name}.txt").open("r") as protein_data_file:
        protein_data = max(protein_data_file.readlines(), key=len)

    if row.mode != row.action:
        triple = f"{row.drug_name} - {row.mode} and {row.action} - {row.protein_name}"
    else:
        triple = f"{row.drug_name} - {row.mode} - {row.protein_name}"
    
    prompt = f"""I am providing you with a drug-protein interaction as a knowledge graph triple along with their associated background information. 
I would like you to write 5 multiple-choice question-answer pairs. Each question must be a created by combining all of the provided information. 
Please structure your output as follows,
Question: <question text>
Choices: <answer choices labelled as (a), (b), (c), (d)>
Correct choice: <among (a), (b), (c) or (d)>

Drug-Protein interaction triple: {triple}

Drug name: {row.drug_name} 
Drug information: {drug_data.strip()}

Protein name: {row.protein_name}
Protein information: {protein_data.strip()}
"""
    break

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.3-70B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

def generate_response(messages):
    input_text = ""
    for message in messages:
        input_text += f"{message['role']}: {message['content']}\n"
    
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

messages = [
    {"role": "system", "content": "You are a question answering AI assistant. You are provided with a question and choices for that question. I would like you tell me what the correct answer is after explaining your reason. Please provide your answer like,\nanswer: a"},
    {"role": "user", "content": """Question: What is the significance of HRH1 in the context of antimicrobial research?
Choices:
(a) HRH1 activation enhances the immune response to bacterial infections.
(b) HRH1 inhibition has been linked to reducing biofilm formation in Staphylococcus aureus.
(c) HRH1 promotes bacterial resistance, making it a target for pro-bacterial therapies.
(d) HRH1 is primarily involved in viral infections and has no relevance to bacteria."""},
]

generate_response(messages)